<a href="https://colab.research.google.com/github/gangasani-anusha/Approximate_Nearest_Neighbour_Search/blob/main/ANN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approximate Nearest Neighbor Search

## LSH

In [169]:
!pip install lshashing

In [170]:
from lshashing import LSHRandom
import numpy as np

# Loading the data
lsh_data = np.random.randint(size = (10, 10), low = 0, high = 50)
lsh_point = np.random.randint(size = (1,10), low = 0, high = 50)

# Applying the LSH appropriate nearest neighbor search
lshashing = LSHRandom(lsh_data, hash_len = 3, num_tables = 2)

# Displaying data
print(lshashing.tables[1].hash_table)
print(lshashing.knn_search(lsh_data, lsh_point[0], k = 3, buckets = 2, radius = 2))

{25: [0, 9], 1: [1, 4], 4: [2], 0: [3, 8], 9: [5, 6], 16: [7]}
[[48.77499359  1.        ]
 [49.71921158  2.        ]
 [51.96152423  3.        ]]


In [171]:
# Loading the data
lsh_data = np.random.randint(size = (10, 10), low = 0, high = 50)
lsh_point = np.random.randint(size = (1, 10), low = 0, high = 50)

# Applying the LSH appropriate nearest neighbor search
lsh_random_parallel = LSHRandom(lsh_data, 4, parallel = True)
lsh_random_parallel.knn_search(lsh_data, lsh_point[0], 5, 2, parallel = True)

[Neighbor(index=9, distance=36.124783736376884, value=[[11  5 42]...]),
 Neighbor(index=8, distance=48.2078831727758, value=[[26 16 42]...]),
 Neighbor(index=7, distance=57.043842787806646, value=[[23 17 33]...]),
 Neighbor(index=6, distance=57.253820833198546, value=[[17 13 23]...]),
 Neighbor(index=3, distance=60.14149981501958, value=[[ 3 23 35]...])]

## Exhaustive Search



In [172]:
# Compute the squared Euclidean distance
def SED(X, Y):
    return sum((i-j)**2 for i, j in zip(X, Y))

SED( (9, 7), (3, 5) )

40

In [173]:
# Calculating the nearest neighbor with the reference points
def exhaustive_search(*, es_points, exhaustive_points):
    return {
        query_p: min(
            exhaustive_points,
            key=lambda X: SED(X, query_p),
        )
        for query_p in es_points
    }

# Defining the data
exhaustive_points = [ (2, 3), (1, 5), (3, 4), (5, 10) ]
es_points = [
    (11, 3), (2, 7), (8, 5), (1, 10), (17, 15), (7, 13), (7, 9)
]

exhaustive_search(
    exhaustive_points = exhaustive_points,
    es_points = es_points,
)

{(1, 10): (5, 10),
 (2, 7): (1, 5),
 (7, 9): (5, 10),
 (7, 13): (5, 10),
 (8, 5): (3, 4),
 (11, 3): (3, 4),
 (17, 15): (5, 10)}

## Product Quantization

In [174]:
!pip install nanopq

In [175]:
# Import libraries
import nanopq
import numpy as np

# Loading the data
value1 = np.random.random((500, 10)).astype(np.float32)
value2 = np.random.random((250, 10)).astype(np.float32)
pq_query_data = np.random.random((10, )).astype(np.float32)

In [176]:
pq_data = nanopq.PQ(M=2, Ks=10, verbose=True)

M: 2, Ks: 10, code_dtype: <class 'numpy.uint8'>


In [177]:
pq_data = nanopq.PQ(M=2, Ks=10).fit(vecs=value2, iter=20, seed=123)

M: 2, Ks: 10, code_dtype: <class 'numpy.uint8'>
iter: 20, seed: 123
Training the subspace: 0 / 2
Training the subspace: 1 / 2


In [178]:
# Vectors encoded to PQ-codes.
value1_code = pq_data.encode(vecs=value1)

Encoding the subspace: 0 / 2
Encoding the subspace: 1 / 2


In [179]:
pq_datatable = pq_data.dtable(query=pq_query_data) 
new_pq = pq_datatable.adist(codes=value1_code) 
new_pq = pq_data.dtable(query=pq_query_data).adist(codes=value1_code) 
min_value = np.argmin(new_pq)

In [180]:
# The results by PQ
print(new_pq[:30])

# The results by the exact scan
new_pq_exact = np.linalg.norm(value1 - pq_query_data, axis=1) ** 2
print(new_pq_exact[:30])

[1.26356    0.9783082  1.2880634  1.3911514  1.5733516  1.4296246
 1.2613258  0.5904324  1.1854758  1.4976234  1.3838279  0.5783453
 0.5783453  1.0990655  1.1045363  1.0777987  1.410969   1.1162719
 0.93581283 1.1957692  1.5360967  0.98010516 1.304497   0.70475554
 0.83476007 0.48798016 1.6675487  1.6402427  1.0990655  0.98308283]
[1.3995913  1.9094465  1.4047843  1.440461   2.3568137  2.0020723
 1.8083136  1.499749   1.6971933  2.1096275  1.1967705  0.6210698
 0.7271497  1.2041146  1.5191325  1.2733145  1.8714951  1.5218028
 2.0556386  1.2519672  1.6659839  1.9409795  2.186497   0.60437876
 1.1371644  1.0938314  1.9385899  1.7647533  1.512447   1.4591714 ]


In [181]:
# Vectors approximately reconstructed by fetching codewords
value1_reconstructed = pq_data.decode(codes=value1_code)
print(value1[:3])
print(value1_reconstructed[:3])

[[0.9599391  0.8767047  0.46805966 0.6259065  0.45718172 0.22294624
  0.376677   0.10388424 0.6665271  0.19203015]
 [0.4754678  0.9674366  0.03166893 0.15172996 0.2985792  0.941807
  0.9088418  0.16200083 0.9811178  0.7507475 ]
 [0.5399771  0.9317029  0.8806071  0.3913165  0.6563432  0.6473851
  0.3269682  0.17939018 0.46680987 0.26328105]]
[[0.74765116 0.7883339  0.5643746  0.7480175  0.43623787 0.23119187
  0.2595143  0.3179935  0.5939355  0.17573178]
 [0.67624027 0.6764091  0.1483006  0.33546132 0.4748308  0.5377833
  0.85469955 0.26338068 0.7303814  0.55986005]
 [0.62674683 0.6114771  0.79226565 0.35940847 0.7483627  0.76050806
  0.31336385 0.28216836 0.24166429 0.34293252]]


## Trees and Graphs

In [182]:
# Importing libraries 
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Loading the data
data = np.array([[-11, -5], [-8, -10], [-7, -1], [5, 3], [4, 1], [9, 8]])

# Finding nearest neighbors with ball tree approach
tg_neighbors = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
tg_distances, tg_indices = tg_neighbors.kneighbors(data)
tg_indices
tg_distances

array([[0.        , 5.65685425],
       [0.        , 5.83095189],
       [0.        , 5.65685425],
       [0.        , 2.23606798],
       [0.        , 2.23606798],
       [0.        , 6.40312424]])

In [183]:
tg_neighbors.kneighbors_graph(data).toarray()

array([[1., 0., 1., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 0., 1.]])

## HNSW

In [184]:
!pip install n2

In [185]:
# Importing Libraries 
import numpy as np
from n2 import HnswIndex

# Loading data
Values, reference = 20240, 20
hnsw_data = np.arange(Values * reference).reshape(Values, reference)

# Applying HNSW approach
hnsw_index_data = HnswIndex(reference)
for data in hnsw_data:
    hnsw_index_data.add_data(data)
hnsw_index_data.build(m=5, n_threads=3)
print(hnsw_index_data.search_by_id(15, 36))

[15, 16, 14, 17, 13, 18, 19, 12, 20, 21, 22, 23, 11, 24, 25, 26, 27, 28, 29, 10, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 9, 42, 43, 44]
